In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim

# Crea una sesión de Spark
spark = SparkSession.builder \
    .appName("Preprocessing Data") \
    .getOrCreate()

# Carga datos desde un archivo CSV
oij_df = spark.read.csv("C:/Users/grana/OneDrive/Escritorio/Bases de datos/spark/data/OIJ.csv", header=True, inferSchema=True)
inec_df = spark.read.csv("C:/Users/grana/OneDrive/Escritorio/Bases de datos/spark/data/INEC.csv", header=True, inferSchema=True)

def eliminar_espacios(dataframe, columna):
    return dataframe.withColumn(columna, trim(dataframe[columna]))

# Elimina espacios en blanco en la columna 'Distrito' en ambos DataFrames
oij_df = eliminar_espacios(oij_df, 'Distrito')
inec_df = eliminar_espacios(inec_df, 'Provincia cantón y distrito')  # Ajustar si la columna tiene otro nombre
# Elimina la columna "_C11' del DataFrame OIJ
oij_df = oij_df.drop('_C11')

# Muestra los resultados
print("Datos del OIJ después de limpiar:")
oij_df.show(50, truncate=False)

print("Datos del INEC después de limpiar:")
inec_df.show(50, truncate=False)



Datos del OIJ después de limpiar:
+------+-----------+----------+-------------------+--------+-----------------------------------+-------------+------+------------+----------+-------------+------------------------+
|Delito|SubDelito  |Fecha     |Hora               |Victima |SubVictima                         |Edad         |Sexo  |Nacionalidad|Provincia |Canton       |Distrito                |
+------+-----------+----------+-------------------+--------+-----------------------------------+-------------+------+------------+----------+-------------+------------------------+
|ASALTO|ARMA BLANCA|1/10/2011 |18:00:00 - 20:59:59|PERSONA |PEATON [PERSONA]                   |Mayor de edad|HOMBRE|NICARAGUA   |SAN JOSE  |ALAJUELITA   |SAN FELIPE              |
|ASALTO|ARMA BLANCA|2/2/2011  |03:00:00 - 05:59:59|PERSONA |PEATON [PERSONA]                   |Mayor de edad|HOMBRE|COSTA RICA  |CARTAGO   |OREAMUNO     |SAN RAFAEL              |
|ASALTO|ARMA BLANCA|10/23/2011|21:00:00 - 23:59:59|PERSONA |P